In [23]:
import pandas as pd
import torch
import torch.nn as nn
from collections import defaultdict, deque
import pickle

In [65]:
import sys
sys.path.insert(0, "../src")   
from model_utils import load_models_from_dir
from model_utils import predict_next_days_for_sids

In [4]:
%load_ext autoreload
%autoreload 2

In [43]:
test_df = pd.read_excel(
    "../output/data/test_top_10_store_10_store_item.xlsx",
)
test_df = test_df.sort_values(by=['date', 'store_item'])
test_df = test_df.reset_index(drop=True)
test_df.head(5)


,date,store_item,store_nbr,item_nbr,onpromotion,unit_sales
0,2017-08-16,3_1047679,3,1047679,0,0
1,2017-08-16,3_114790,3,114790,1,0
2,2017-08-16,3_305229,3,305229,0,0
3,2017-08-16,3_314384,3,314384,1,0
4,2017-08-16,3_364606,3,364606,0,0


In [44]:
test_df["date"].nunique()
test_df["date"].min(), test_df["date"].max()

(Timestamp('2017-08-16 00:00:00'), Timestamp('2017-08-31 00:00:00'))

In [45]:

def preprocess_test_df(df, feature_cols):
    df['onpromotion'] = df['onpromotion'].astype(bool).astype(int)
    df[feature_cols] = df[feature_cols].astype('float32')
    df["date"] = pd.to_datetime(df["date"])
#    df["date"] = df["date"].dt.strftime("%Y-%m-%d")
    df.sort_values("date", inplace=True)
    df.reset_index(drop=True, inplace=True)
    df["store_item"] = df["store_item"].astype(str)
    df.rename(columns={"store_nbr": "store", "item_nbr": "item"}, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df




In [46]:
test_df = preprocess_test_df(test_df, ["unit_sales"])
test_df.dtypes

date           datetime64[ns]
store_item             object
store                   int64
item                    int64
onpromotion             int64
unit_sales            float32
dtype: object

In [47]:
test_df.head(5)

,date,store_item,store,item,onpromotion,unit_sales
0,2017-08-16,3_1047679,3,1047679,0,0.0
1,2017-08-16,50_305229,50,305229,0,0.0
2,2017-08-16,50_114790,50,114790,1,0.0
3,2017-08-16,50_1047679,50,1047679,0,0.0
4,2017-08-16,49_584028,49,584028,0,0.0


In [59]:
test_df.query("store_item == '3_1047679'")

,date,store_item,store,item,onpromotion,unit_sales
0,2017-08-16,3_1047679,3,1047679,0,0.0
175,2017-08-17,3_1047679,3,1047679,0,0.0
271,2017-08-18,3_1047679,3,1047679,0,0.0
372,2017-08-19,3_1047679,3,1047679,0,0.0
472,2017-08-20,3_1047679,3,1047679,0,0.0
572,2017-08-21,3_1047679,3,1047679,0,0.0
671,2017-08-22,3_1047679,3,1047679,0,0.0
772,2017-08-23,3_1047679,3,1047679,0,0.0
872,2017-08-24,3_1047679,3,1047679,0,0.0
972,2017-08-25,3_1047679,3,1047679,0,0.0


In [31]:
train_df = pd.read_excel("../output/data/20250529_scaled_train_top_10_store_10_item_sales_cyclical_features_16_days_X_y.xlsx")
train_df.head(5)

,start_date,store_item,store,item,sales_day_1,sales_day_2,sales_day_3,sales_day_4,sales_day_5,sales_day_6,...,y_monthofyear_cos_7,y_monthofyear_cos_8,y_monthofyear_cos_9,y_monthofyear_cos_10,y_monthofyear_cos_11,y_monthofyear_cos_12,y_monthofyear_cos_13,y_monthofyear_cos_14,y_monthofyear_cos_15,y_monthofyear_cos_16
0,2013-01-09,3_1047679,3,1047679,0.326480,0.129257,0.457841,0.563467,0.169148,0.289882,...,0.933013,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75
1,2013-01-25,3_1047679,3,1047679,0.183388,0.129257,0.399663,0.155418,0.065156,0.178771,...,0.750000,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75
2,2013-02-10,3_1047679,3,1047679,0.200658,0.076560,0.204890,0.177090,0.049838,0.165736,...,0.500000,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
3,2013-02-26,3_1047679,3,1047679,0.141447,0.087497,0.194772,0.180805,0.113269,0.456859,...,0.500000,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
4,2013-03-14,3_1047679,3,1047679,0.351974,0.104151,0.424958,0.385759,0.063215,0.155183,...,0.250000,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25,0.25


In [33]:
# Get the latest row per store_item
idx = train_df.groupby("store_item")["start_date"].idxmax()
last_date_df = train_df.loc[idx].reset_index(drop=True)

# Identify y_ columns
y_cols = [col for col in train_df.columns if col.startswith("y_")]

# Meta columns to carry over
meta_cols = ["start_date", "store_item", "store", "item"]

# Build new rows
new_rows = []

for _, row in last_date_df.iterrows():
    new_X = {col.replace("y_", "", 1): row[col] for col in y_cols}  # only first "y_" is removed
    new_y = {col: 0.0 for col in y_cols}
    meta = {col: row[col] for col in meta_cols}
    meta["start_date"] = pd.to_datetime(meta["start_date"]) + pd.Timedelta(days=16)
    new_row = {**meta, **new_X, **new_y}
    new_rows.append(new_row)

# Final DataFrame with new rows
last_date_df = pd.DataFrame(new_rows)
last_date_df


,start_date,store_item,store,item,sales_day_1,sales_day_2,sales_day_3,sales_day_4,sales_day_5,sales_day_6,...,y_monthofyear_cos_7,y_monthofyear_cos_8,y_monthofyear_cos_9,y_monthofyear_cos_10,y_monthofyear_cos_11,y_monthofyear_cos_12,y_monthofyear_cos_13,y_monthofyear_cos_14,y_monthofyear_cos_15,y_monthofyear_cos_16
0,2017-07-31,3_1047679,3,1047679,0.332237,0.073328,0.358347,0.319505,0.100755,0.274984,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-07-31,3_114790,3,114790,0.015625,0.004226,0.013491,0.013622,0.005178,0.019243,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-07-31,3_305229,3,305229,0.034539,0.015909,0.046374,0.021672,0.009709,0.034140,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-07-31,3_314384,3,314384,0.059211,0.012926,0.083474,0.047678,0.019417,0.052142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-07-31,3_364606,3,364606,0.057566,0.022123,0.107926,0.046440,0.021791,0.052142,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2017-07-31,8_502331,8,502331,0.021382,0.011683,0.030354,0.014861,0.008846,0.021726,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,2017-07-31,8_567623,8,567623,0.013158,0.002486,0.013491,0.004954,0.002157,0.011794,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,2017-07-31,8_581078,8,581078,0.007401,0.003977,0.011804,0.004334,0.003020,0.010552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,2017-07-31,8_582864,8,582864,0.024513,0.007462,0.033646,0.025017,0.005979,0.024430,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
print("Unique (store, item) pairs:", last_date_df["store_item"].nunique())

Unique (store, item) pairs: 100


In [37]:
models = load_models_from_dir()

In [ ]:

with open('../output/data/20250529_top10_store_10_item_minmax_y_scaler.pkl', 'rb') as f:
    y_scaler = pickle.load(f)

In [71]:
# --- Extract model and feature columns ---
sid = "49_584028"
model, feature_cols = models[sid]
model.eval()

# --- Prepare input ---
input_data = last_date_df.query("store_item == @sid")
x = input_data[feature_cols].values.astype("float32")
x_tensor = torch.tensor(x)

# --- Make prediction ---
with torch.no_grad():
    y_pred = model(x_tensor)

# --- Convert prediction to DataFrame with feature column names ---
y_pred_scaled = pd.DataFrame(y_pred.numpy(), columns=feature_cols)
y_pred_df = pd.DataFrame(y_scaler.inverse_transform(y_pred_scaled), columns=feature_cols)

# --- Keep only 'sales_day_' columns ---
sales_day_cols = [col for col in y_pred_df.columns if col.startswith("sales_day_")]
sales_pred_df = y_pred_df[sales_day_cols]

# --- Extract metadata ---
meta = input_data.iloc[0][["store_item", "store", "item"]].to_dict()
start_date = pd.to_datetime(input_data.iloc[0]["start_date"]) + pd.Timedelta(days=15)
# --- Build final output: one row per predicted day ---
rows = []
for i, col in enumerate(sales_day_cols):
    row = {
        "date": start_date + pd.Timedelta(days=i + 1),
        **meta,  # Add metadata first
        "unit_sales": sales_pred_df.at[0, col]  # Add 'sales' last
    }
    rows.append(row)

# --- Final DataFrame ---
final_df = pd.DataFrame(rows)
final_df


,date,store_item,store,item,unit_sales
0,2017-08-16,49_584028,49,584028,-0.792584
1,2017-08-17,49_584028,49,584028,-0.791266
2,2017-08-18,49_584028,49,584028,-0.791710
3,2017-08-19,49_584028,49,584028,-0.794866
4,2017-08-20,49_584028,49,584028,-0.790937
5,2017-08-21,49_584028,49,584028,-0.792082
6,2017-08-22,49_584028,49,584028,-0.789120
7,2017-08-23,49_584028,49,584028,-0.792353
8,2017-08-24,49_584028,49,584028,-0.795245
9,2017-08-25,49_584028,49,584028,-0.795531


In [61]:
final_df.dtypes

date          datetime64[ns]
store_item            object
store                  int64
item                   int64
unit_sales           float32
dtype: object

In [62]:
test_df.dtypes

date           datetime64[ns]
store_item             object
store                   int64
item                    int64
onpromotion             int64
unit_sales            float32
dtype: object

In [72]:

# Step 2: Merge with test_df using a left join

merged_df = test_df.query("store_item == @sid").merge(
    final_df,
    on=["date", "store_item", "store", "item"],
    how="left",
    suffixes=("", "_pred")
)

# Step 3: Replace test_df unit_sales with predicted ones where available
merged_df["unit_sales"] = merged_df["unit_sales_pred"].combine_first(merged_df["unit_sales"])

# Step 4: Drop the helper column
merged_df = merged_df.drop(columns=["unit_sales_pred"])
merged_df


,date,store_item,store,item,onpromotion,unit_sales
0,2017-08-16,49_584028,49,584028,0,-0.792584
1,2017-08-17,49_584028,49,584028,0,-0.791266
2,2017-08-18,49_584028,49,584028,1,-0.791710
3,2017-08-19,49_584028,49,584028,0,-0.794866
4,2017-08-20,49_584028,49,584028,0,-0.790937
5,2017-08-21,49_584028,49,584028,0,-0.792082
6,2017-08-22,49_584028,49,584028,0,-0.789120
7,2017-08-23,49_584028,49,584028,0,-0.792353
8,2017-08-24,49_584028,49,584028,0,-0.795245
9,2017-08-25,49_584028,49,584028,1,-0.795531


In [73]:
all_preds_df = predict_next_days_for_sids(last_date_df, models, y_scaler)
all_preds_df.head()


,date,store_item,store,item,unit_sales
0,2017-08-16,45_1047679,45,1047679,0.133876
1,2017-08-17,45_1047679,45,1047679,0.279128
2,2017-08-18,45_1047679,45,1047679,0.288430
3,2017-08-19,45_1047679,45,1047679,0.095799
4,2017-08-20,45_1047679,45,1047679,0.499540


In [74]:
# Step 2: Merge with test_df using a left join
merged_df = test_df.merge(
    all_preds_df,
    on=["date", "store_item", "store", "item"],
    how="left",
    suffixes=("", "_pred")
)

# Step 3: Replace test_df unit_sales with predicted ones where available
merged_df["unit_sales"] = merged_df["unit_sales_pred"].combine_first(merged_df["unit_sales"])

# Step 4: Drop the helper column
merged_df = merged_df.drop(columns=["unit_sales_pred"])
merged_df

,date,store_item,store,item,onpromotion,unit_sales
0,2017-08-16,3_1047679,3,1047679,0,338.339172
1,2017-08-16,50_305229,50,305229,0,-0.802054
2,2017-08-16,50_114790,50,114790,1,-0.801325
3,2017-08-16,50_1047679,50,1047679,0,30.347908
4,2017-08-16,49_584028,49,584028,0,-0.792584
...,...,...,...,...,...,...
1595,2017-08-31,45_581078,45,581078,0,41.948650
1596,2017-08-31,45_567623,45,567623,0,-0.792051
1597,2017-08-31,45_502331,45,502331,0,-0.785579
1598,2017-08-31,46_502331,46,502331,0,-0.793558


In [75]:
merged_df.query("store_item == '3_1047679'")
    

,date,store_item,store,item,onpromotion,unit_sales
0,2017-08-16,3_1047679,3,1047679,0,338.339172
175,2017-08-17,3_1047679,3,1047679,0,412.000946
271,2017-08-18,3_1047679,3,1047679,0,388.579803
372,2017-08-19,3_1047679,3,1047679,0,522.115356
472,2017-08-20,3_1047679,3,1047679,0,592.439270
572,2017-08-21,3_1047679,3,1047679,0,453.254120
671,2017-08-22,3_1047679,3,1047679,0,446.928162
772,2017-08-23,3_1047679,3,1047679,0,378.550537
872,2017-08-24,3_1047679,3,1047679,0,271.746216
972,2017-08-25,3_1047679,3,1047679,0,500.431335


In [77]:
merged_df.query("store_item == '50_1047679'")


,date,store_item,store,item,onpromotion,unit_sales
3,2017-08-16,50_1047679,50,1047679,0,30.347908
106,2017-08-17,50_1047679,50,1047679,0,134.643295
205,2017-08-18,50_1047679,50,1047679,0,44.890320
302,2017-08-19,50_1047679,50,1047679,0,52.235256
403,2017-08-20,50_1047679,50,1047679,0,112.364883
505,2017-08-21,50_1047679,50,1047679,0,77.546318
606,2017-08-22,50_1047679,50,1047679,0,34.292873
702,2017-08-23,50_1047679,50,1047679,0,28.732626
805,2017-08-24,50_1047679,50,1047679,0,30.620260
909,2017-08-25,50_1047679,50,1047679,0,33.899410
